<a href="https://colab.research.google.com/github/AfshinRezakhani/Thesis1/blob/main/GAN_GRU_UCI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Reshape
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

file_path = "/content/UCI.csv"
df = pd.read_csv(file_path)

target_column = "A"
features = df.drop(columns=[target_column]).values
target = df[target_column].values

scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

def build_generator():
    model = Sequential([
        GRU(128, return_sequences=True, input_shape=(1, X_train.shape[2])),
        Dropout(0.2),
        GRU(64, return_sequences=True),
        Dropout(0.2),
        GRU(32, return_sequences=False),
        Dense(X_train.shape[2]),
        Reshape((1, X_train.shape[2]))
    ])
    return model

def build_discriminator():
    model = Sequential([
        GRU(128, return_sequences=True, input_shape=(1, X_train.shape[2])),
        Dropout(0.3),
        GRU(64, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    return model

def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential([generator, discriminator])
    return model

generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

discriminator.compile(loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5), metrics=["accuracy"])
gan.compile(loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5))

epochs = 2000
batch_size = 32
half_batch = batch_size // 2

for epoch in range(epochs):
    noise = np.random.normal(0, 1, (half_batch, 1, X_train.shape[2]))
    generated_data = generator.predict(noise)

    idx = np.random.randint(0, X_train.shape[0], half_batch)
    real_data = X_train[idx]

    X_combined = np.concatenate((real_data, generated_data))
    y_combined = np.concatenate((np.ones((half_batch, 1)), np.zeros((half_batch, 1))))

    d_loss = discriminator.train_on_batch(X_combined, y_combined)

    noise = np.random.normal(0, 1, (batch_size, 1, X_train.shape[2]))
    y_mislabeled = np.ones((batch_size, 1))

    g_loss = gan.train_on_batch(noise, y_mislabeled)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Discriminator Loss: {d_loss[0]:.4f}, Generator Loss: {g_loss:.4f}")

num_samples = len(X_train)
noise = np.random.normal(0, 1, (num_samples, 1, X_train.shape[2]))
generated_samples = generator.predict(noise)

synthetic_labels = np.ones((num_samples,))

X_balanced = np.concatenate((X_train.squeeze(), generated_samples.squeeze()))
y_balanced = np.concatenate((y_train, synthetic_labels))

df_balanced = pd.DataFrame(X_balanced)
df_balanced["A"] = y_balanced
df_balanced.to_csv("/content/UCI_balanced_WithGRU.csv", index=False)
print("✅ Balanced dataset saved as UCI_balanced_WithGRU.csv")

y_pred_probs = discriminator.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"✅ Accuracy: {accuracy:.4f}")
print(f"✅ F1-Score: {f1:.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 767ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0: Discriminator Loss: 0.6902, Generator Loss: 0.6931
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 